## Setup

### Imports

In [2]:
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

2021-10-05 23:01:03.245240: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-05 23:01:03.245264: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Constants

In [8]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE # ??
BATCH_SIZE = 1024           # ??
BUFFER_SIZE = 10000         # ??

path_to_file = 'shakespeare.txt'
num_ns = 4                  # ??
vocab_size = 4096           # ??
sequence_length = 10        # ??

# Generate Training Data
    A tuple (target, context, label) tensors constitutes one training example
_(In skip-gram negative sampling Word2Vec model)_

## Main function

- Batch 1 positive `context_word` and `num_ns` negative context words into one tensor;
- This produces a set of positive skip-grams _(labelled as 1)_ & negative samples _(labelled as 0)_ for each target word;
- Notice that the target is of shape (1,) while the context and label are of shape (1 + `num_ns`,)

In [5]:
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  '''
    - Generate training examples for the Word2Vec model;
    - Iterates over each word from each sequence to collect positive and negative context words;
    - Length of target, contexts and labels should be same, representing the total number of training examples;
    - Generates skip-gram pairs with negative sampling for a list of sequences (int-encoded sentences)
    based on window size, number of negative samples and vocabulary size.
  '''

  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for vocab_size tokens (word frequency rank).
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0)

    # Iterate over each positive skip-gram pair to produce training examples
    # with positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)

      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=SEED,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      negative_sampling_candidates = tf.expand_dims(
          negative_sampling_candidates, 1)

      context = tf.concat([context_class, negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

## Prepare

### Custom text standardization function

In [7]:
def custom_standardization(input_data: str) -> str:
  '''
    Custom standardization function: Lowercase text & remove punctuation.
  '''

  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase, '[%s]' % re.escape(string.punctuation), '')

### Vectorize sentences

- `TextVectorization` layer normalize, split, and map strings to integers;
- `adapt()` update the layer state to represent the text corpus;
- After that vocabulary can be accessed with `get_vocabulary()` _(This function returns a list of all vocabulary tokens sorted (descending) by their frequency)_;        
- After that, `vectorize_layer` can be used to generate a `tf.data.Dataset` of integer encoded sentences;
- `text_vector_ds` is a `tf.data.Dataset` with vectors for each element in the Dataset _(`text_ds`)_;
- From `text_vector_ds` we generate `sequences`;
- `sequences` is the dataset flattened as a list of int encoded sentence vectors _(It's required as we'd iterate over each sentence in the DS to produce positive & negative examples)_;

In [ ]:
# Read Dataset from file
with open(path_to_file) as f: 
  lines = f.read().splitlines()
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

# Map strings to integers
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length # Pad all samples to same length
  )

# Create vocabulary
vectorize_layer.adapt(text_ds.batch(1024)) 
inverse_vocab = vectorize_layer.get_vocabulary()

# Generate list of int sequence vectors of sentenses
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()
sequences = list(text_vector_ds.as_numpy_iterator())

### Generate training examples from sequences

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED
)

targets = np.array(targets)
contexts = np.array(contexts)[:,:,0]
labels = np.array(labels)

# print('\n')
# print(f"targets.shape: {targets.shape}")
# print(f"contexts.shape: {contexts.shape}")
# print(f"labels.shape: {labels.shape}")

### Performance tweaks

- For performance reasons we go back with our data into the `tf.data.Dataset` format;
- It's better to perform efficient batching for the potentially large number of training examples;
- `cache()` and `prefetch()` improve performance;

In [9]:
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)

NameError: name 'targets' is not defined

# Modeling